In [1]:
import os
import time
import numpy as np
import pandas as pd
import tensorflow as tf

#from tensorflow.models.rnn.seq2seq import sequence_loss_by_example

from DropoutWrapper import DropoutWrapper
from MultiRNNCell2 import MultiRNNCell2

# parses the dataset
import ptb_reader

%autosave 0

Autosave disabled


In [2]:
!pwd

/notebooks/1A_Tensorflow/AA_RNN_Practice/TF_RNN_1/my_lstm_odyssey


In [3]:
# define artifact directories where results from the session can be saved
model_path = os.environ.get('MODEL_PATH', 'models/')
checkpoint_path = os.environ.get('CHECKPOINT_PATH', 'checkpoints/')
summary_path = os.environ.get('SUMMARY_PATH', 'logs/')

In [4]:
model_path

'models/'

In [5]:
!pwd

/notebooks/1A_Tensorflow/AA_RNN_Practice/TF_RNN_1/my_lstm_odyssey


In [6]:
# load dataset
train_data, valid_data, test_data, _ = ptb_reader.ptb_raw_data(data_path="ptb")

def write_csv(arr, path):
    df = pd.DataFrame(arr)
    df.to_csv(path)

In [7]:
type(train_data)

list

In [8]:
#0.12.1 version
def sequence_loss_by_example(logits, targets, weights,
                             average_across_timesteps=True,
                             softmax_loss_function=None, name=None):
  """Weighted cross-entropy loss for a sequence of logits (per example).

  Args:
    logits: List of 2D Tensors of shape [batch_size x num_decoder_symbols].
    targets: List of 1D batch-sized int32 Tensors of the same length as logits.
    weights: List of 1D batch-sized float-Tensors of the same length as logits.
    average_across_timesteps: If set, divide the returned cost by the total
      label weight.
    softmax_loss_function: Function (inputs-batch, labels-batch) -> loss-batch
      to be used instead of the standard softmax (the default if this is None).
    name: Optional name for this operation, default: "sequence_loss_by_example".

  Returns:
    1D batch-sized float Tensor: The log-perplexity for each sequence.

  Raises:
    ValueError: If len(logits) is different from len(targets) or len(weights).
  """
  if len(targets) != len(logits) or len(weights) != len(logits):
    raise ValueError("Lengths of logits, weights, and targets must be the same "
                     "%d, %d, %d." % (len(logits), len(weights), len(targets)))
  with tf.name_scope(name, "sequence_loss_by_example",
                      logits + targets + weights):
    log_perp_list = []
    for logit, target, weight in zip(logits, targets, weights):
      if softmax_loss_function is None:
        # TODO(irving,ebrevdo): This reshape is needed because
        # sequence_loss_by_example is called with scalars sometimes, which
        # violates our general scalar strictness policy.
        target = tf.reshape(target, [-1])
        crossent = tf.nn.sparse_softmax_cross_entropy_with_logits(
            logits=logit, labels=target)
      else:
        crossent = softmax_loss_function(logit, target)
      log_perp_list.append(crossent * weight)
    log_perps = tf.add_n(log_perp_list)
    if average_across_timesteps:
      total_size = tf.add_n(weights)
      total_size += 1e-12  # Just to avoid division by 0 for all-0 weights.
      log_perps /= total_size
  return log_perps

In [9]:
#from tf.models.rnn.seq2seq import sequence_loss_by_example

class PTBModel(object):
    def __init__(self, CellType, is_training, config):
        self.batch_size = batch_size = config.batch_size
        self.num_steps = num_steps = config.num_steps
        size = config.hidden_size
        vocab_size = config.vocab_size
        
        self.input_data = tf.placeholder(tf.int32, [batch_size, num_steps], name="input_data")
        self.targets = tf.placeholder(tf.int32, [batch_size, num_steps], name="targets")
        
        lstm_cell = CellType(size)
        if is_training and config.keep_prob < 1:
            lstm_cell = tf.contrib.rnn.DropoutWrapper(lstm_cell, output_keep_prob = config.keep_prob)
        cell = tf.contrib.rnn.MultiRNNCell([lstm_cell] * config.num_layers)
        self.initial_state = cell.zero_state(batch_size, tf.float32)
        
        # initalizer used for reusable variable initializer (see 'get_variable')
        initializer = tf.random_uniform_initializer(-config.init_scale, config.init_scale)
        
#        with tf.device("/cpu:0"):
        embedding = tf.get_variable("embedding", [vocab_size, size], initializer=initializer)
        inputs = tf.nn.embedding_lookup(embedding, self.input_data)
            
        if is_training and config.keep_prob < 1:
            inputs = tf.nn.dropout(inputs, config.keep_prob)
            
        outputs = []
        states = []
        state = self.initial_state
        
        with tf.variable_scope("RNN", initializer=initializer):
            for time_step in range(num_steps):
                if time_step > 0:
                    tf.get_variable_scope().reuse_variables()
                    
                inputs_slice = inputs[:,time_step,:]
                (cell_output, state) = cell(inputs_slice, state)
                
                outputs.append(cell_output)
                states.append(state)
                
        self.final_state = states[-1]
        
        output = tf.reshape(tf.concat(outputs,1), [-1, size])
        w = tf.get_variable("softmax_w",
                           [size, vocab_size],
                           initializer=initializer)
        b = tf.get_variable("softmax_b", [vocab_size], initializer=initializer)
        
        logits = tf.nn.xw_plus_b(output, w, b) # compute logits for loss
        targets = tf.reshape(self.targets, [-1]) # reshape target outputs
        weights = tf.ones([batch_size * num_steps]) # used to scale the loss average
        
        # computes loss and performs softmax on our fully-connected output layer
#         def sequence_loss_by_example(inputs, targets, weights, loss_function,
#                              average_across_timesteps=True, name=None):
        
        loss = sequence_loss_by_example([logits], [targets], [weights], vocab_size)
        self.cost = cost = tf.div(tf.reduce_sum(loss), batch_size, name="cost")
        
        if is_training:
            # set up learning rate variable to decay
            self.lr = tf.Variable(0.98, trainable=False)
            #self.lr = tf.Variable(0.0001, trainable=False)
            
            # define training operation and clip the gradients
            tvars = tf.trainable_variables()
            grads, _ = tf.clip_by_global_norm(tf.gradients(cost, tvars), config.max_grad_norm)
            
            optimizer = tf.train.GradientDescentOptimizer(self.lr)
            #optimizer = tf.train.AdamOptimizer(learning_rate=self.lr)
            #optimizer = tf.train.RMSPropOptimizer(learning_rate=self.lr)
            
            self.train_op = optimizer.apply_gradients(zip(grads, tvars), name="train")
        else:
            # if this model isn't for training (i.e. testing/validation) then we don't do anything here
            self.train_op = tf.no_op()

In [10]:
def run_epoch(sess, model, data, verbose=False):
    epoch_size = ((len(data) // model.batch_size) - 1) // model.num_steps
    start_time = time.time()
    
    # accumulated counts
    costs = 0.0
    iters = 0
    
    # initial RNN state
    # print(model.initial_state,type(model.initial_state))
    # state = model.initial_state.eval()
    state = tf.get_default_session().run(model.initial_state)
    
    # example of how to change:
    # state = cell.zero_state(batchsize, tf.float32).eval()
    # state = tf.get_default_session().run(cell.zero_state(batchsize, tf.float32))
    
    for step, (x,y) in enumerate(ptb_reader.ptb_iterator(data, model.batch_size, model.num_steps)):
        cost, state, _ = sess.run([model.cost, model.final_state, model.train_op], feed_dict={
            model.input_data: x,
            model.targets: y,
            model.initial_state: state
        })
        costs += cost
        iters += model.num_steps
        
        perplexity = np.exp(costs / iters)
        
        if verbose and step %10 == 0:
            progress = (step / epoch_size) * 100
            wps = iters * model.batch_size / (time.time() - start_time)
            print("Progress:  %.1f%% Perpexity: %.2f (Cost: %.2f) Speed: %.0f wps" % (progress, perplexity, cost, wps))
            
        return (costs / iters), perplexity

In [11]:
class Config(object):
    batch_size = 20
    num_steps = 35 # number of unrolled time steps
    hidden_size = 450 # number of blocks in an LSTM cell
    vocab_size = 10000
    max_grad_norm = 5 # maximum gradient for clipping
    init_scale = 0.05 # scale between -0.1 and 0.1 for all random initialization
    keep_prob = 0.5 # dropout probability
    num_layers = 2 # number of LSTM layers
    #learning_rate = 0.0001 # 1.0
    learning_rate = 0.001 # not getting used at the moment
    lr_decay = 0.985
    lr_decay_epoch_offset = 6 # don't decay until after the Nth epoch

In [12]:
# default settings for training
train_config = Config()

# our evaluation runs (validation and testing), use a batch size and time_step of one
eval_config = Config()
eval_config.batch_size = 1
eval_config.num_steps = 1

# number of epochs to perform over the training data
num_epochs = 39

import vanilla, cifg

cell_types = {
    'vanilla': vanilla.VanillaLSTMCell,
#     'nig': NIGLSTMCell,
#     'nfg': NFGLSTMCell,
#     'nog': NOGLSTMCell,
#     'niaf': NIAFLSTMCell,
#     'noaf': NOAFLSTMCell,
#     'np': NPLSTMCell,
     'cifg': cifg.CIFGLSTMCell,
#     'fgr': FGRLSTMCell,
}

#model_name = "vanilla"
model_name = "cifg"

CellType = cell_types[model_name]

In [13]:
with tf.Graph().as_default(), tf.Session() as sess:
    # define our training model
    with tf.variable_scope("model", reuse=None):
        train_model = PTBModel(CellType, is_training=True, config=train_config)
        
    # we create a separate model for validation and testing to alter the batch_size and time steps
    # reuse = True reuses variables from the previously defined 'train_model'
    with tf.variable_scope("model", reuse=True):
        valid_model = PTBModel(CellType, is_training=False, config=train_config)
        test_model = PTBModel(CellType, is_training=False, config=eval_config)

    # create a saver instance to restore from the checkpoint
    saver = tf.train.Saver(max_to_keep=5)

    # initialize our variables
    sess.run(tf.global_variables_initializer())

    # save the graph definition as a protobuf file
    tf.train.write_graph(sess.graph_def, model_path, '%s.pb'.format(model_name), as_text=False)

    train_costs = []
    train_perps = []
    valid_costs = []
    valid_perps = []
    
    # set initial learning rate - can I do this here? so far throws an error
    #train_model.lr = train_config.learning_rate
    #train_model.lr = tf.Variable(train_config.learning_rate, trainable=False)

    for i in range(num_epochs):
        # decay learning rate
        if i > train_config.lr_decay_epoch_offset:
            train_model.lr = train_model.lr * train_config.lr_decay
        
        print("Epoch: %d Learning Rate: %.5f" % (i + 1, sess.run(train_model.lr)))

        # run training pass
        train_cost, train_perp = run_epoch(sess, train_model, train_data, verbose=True)
        print("Epoch: %i Training Perplexity: %.2f (Cost: %.2f)" % (i + 1,  train_perp, train_cost))
        train_costs.append(train_cost)
        train_perps.append(train_perp)

        # run validation pass
        valid_cost, valid_perp = run_epoch(sess, valid_model, valid_data)
        print("Epoch: %i Validation Perplexity: %.2f (Cost: %.2f)" % (i + 1, valid_perp, valid_cost))
        valid_costs.append(valid_cost)
        valid_perps.append(valid_perp)

        # save(self, sess, save_path, global_step, latest_filename, meta_graph_suffix, write_meta_graph, write_state)
        saver.save(sess, checkpoint_path + 'checkpoint', global_step=i,latest_filename=None)

    # run test pass
    test_cost, test_perp = run_epoch(sess, test_model, test_data)
    print("Test Perplexity: %.2f (Cost: %.2f)" % (test_perp, test_cost))

    write_csv(train_costs, os.path.join(summary_path, "train_costs.csv"))
    write_csv(train_perps, os.path.join(summary_path, "train_perps.csv"))
    write_csv(valid_costs, os.path.join(summary_path, "valid_costs.csv"))
    write_csv(valid_perps, os.path.join(summary_path, "valid_perps.csv"))

Epoch: 1 Learning Rate: 0.98000
0.0% Perpexity: 10045.00 (Cost: 322.52) Speed: 214 wps
Epoch: 1 Training Perplexity: 10045.00 (Cost: 9.21)
Epoch: 1 Validation Perplexity: 5833.05 (Cost: 8.67)
Epoch: 2 Learning Rate: 0.98000
0.0% Perpexity: 5584.91 (Cost: 301.97) Speed: 331 wps
Epoch: 2 Training Perplexity: 5584.91 (Cost: 8.63)
Epoch: 2 Validation Perplexity: 20735.91 (Cost: 9.94)
Epoch: 3 Learning Rate: 0.98000
0.0% Perpexity: 16301.92 (Cost: 339.47) Speed: 365 wps
Epoch: 3 Training Perplexity: 16301.92 (Cost: 9.70)
Epoch: 3 Validation Perplexity: 5898.30 (Cost: 8.68)
Epoch: 4 Learning Rate: 0.98000
0.0% Perpexity: 6351.15 (Cost: 306.47) Speed: 370 wps
Epoch: 4 Training Perplexity: 6351.15 (Cost: 8.76)
Epoch: 4 Validation Perplexity: 3753.97 (Cost: 8.23)
Epoch: 5 Learning Rate: 0.98000
0.0% Perpexity: 3640.75 (Cost: 287.00) Speed: 359 wps
Epoch: 5 Training Perplexity: 3640.75 (Cost: 8.20)
Epoch: 5 Validation Perplexity: 8123.57 (Cost: 9.00)
Epoch: 6 Learning Rate: 0.98000
0.0% Perpexit